In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
# imports
import rasterio
from matplotlib import pyplot
import time
# import gdal 
from osgeo import osr, gdal
from PIL import Image
from PIL.TiffTags import TAGS
from os.path import *
import json
from deepforest import main, utilities, get_data, preprocess
Image.MAX_IMAGE_PIXELS = None
import pandas as pd
import geopandas as gpd

c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
!pip install pycuda
import torch
import pycuda.driver as cuda
cuda.init()
## Get Id of default device
torch.cuda.current_device()
# 0
cuda.Device(0).name() # '0' is the id of your GPU


[notice] A new release of pip available: 22.3.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


'NVIDIA GeForce GTX 1080 Ti'

In [8]:
src_img =r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\1297-Makkah-Environmental-Sustainability-Strategy\02-Data\02-satellite\Makkah-WV2-ST-2023_I602488\Makkah-WV2-ST-2023-MOS\23JAN19075627-S2AM-Makkah-WV2-ST-2023-tiled.TIF"
dst_img= r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\1297-Makkah-Environmental-Sustainability-Strategy\02-Data\03-poc\23JAN19075627-S2AM-Makkah-WV2-ST-2023.TIF"

# check if the files exist input file
print(exists(src_img))
print(exists(dst_img))
# img = Image.open(src_img)
# print(img.getbands())

with rasterio.open(src_img) as src:
    for ji, window in src.block_windows(1):
        r = src.read(1, window=window)
        print(r.shape)
        break

# rasterio
src = rasterio.open(src_img)
transform = src.transform 
crs = src.crs

print(crs)
print(src.res)
# print(img.getbands())
# print(img.size)

True
False
(512, 512)
EPSG:32637
(0.46, 0.46)


In [11]:
# get training bonding box and convert to the right format
ano_shp = r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\1297-Makkah-Environmental-Sustainability-Strategy\02-Data\03-poc\mekka-training-tree.shp"
annotations=utilities.shapefile_to_annotations(ano_shp, src_img,  savedir='.')

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_BAG.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_FITS.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_GMT.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_HDF4.dll
127: The specified procedure could not be found.

Can't load requested DLL: C:\Program Files\GDAL\gdalplugins\gdal_HDF4.dll
127: The specified procedure could not be found.

Can't load r

In [12]:
# split in training and testing
train = annotations.sample(frac = 0.75)
test = annotations.drop(train.index)

In [14]:
train_path=r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\1297-Makkah-Environmental-Sustainability-Strategy\02-Data\03-poc\bbox\mekka-training-tree-train.csv"
test_path=r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\1297-Makkah-Environmental-Sustainability-Strategy\02-Data\03-poc\bbox\mekka-training-tree-test.csv"

train.to_csv(train_path)
test.to_csv(test_path)

In [15]:
#Write csv to file and crop

tmpdir = r'D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\1297-Makkah-Environmental-Sustainability-Strategy\02-Data\03-poc\bbox\crop'
# annotations.to_csv("{}/example.csv".format(tmpdir), index=False)
annotations_file = preprocess.split_raster(path_to_raster=src_img,
                                           annotations_file=train_path,
                                           base_dir=tmpdir,
                                           patch_size=500,
                                           patch_overlap=0.25)


c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\preprocess.py:92: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_annotations.image_path = "{}_{}.png".format(image_basename, index)
c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\preprocess.py:110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_annotations.xmax = (selected_annotations.xmin - window_xmin) + (
c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\preprocess.py:112: SettingWithCopyWarning

TypeError: Cannot handle this data type: (1, 1, 3), <i2

In [19]:
del(annotations_file['Unnamed: 0'])
assert annotations_file.shape[1] == 6

In [20]:
ano_split=get_data(r"D:\Dropbox\P.Robeck\BPLA Dropbox\03 Planning\1297-Makkah-Environmental-Sustainability-Strategy\02-Data\03-poc\bbox\crop\23JAN19075627-S2AM-Makkah-WV2-ST-2023.csv")
exists(ano_split)


True

In [21]:
model = main.deepforest()
model.use_release()

# Example run with short training
model.config["train"]['epochs'] = 5
model.config['gpus'] = '-1' #move to GPU and use all the GPU resources
# model.config['cpu'] = 'accelerator'
model.config["save-snapshot"] = False
model.config["train"]["csv_file"] = ano_split
model.config["train"]["root_dir"] = dirname(ano_split)
model.config['num_worker']=16
# model.config['max_epochs']=10
model.create_trainer()

Reading config file: c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\deepforest\data\deepforest_config.yml
Model from DeepForest release https://github.com/weecology/DeepForest/releases/tag/1.0.0 was already downloaded. Loading model from file.


c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\pytorch_lightning\trainer\connectors\accelerator_connector.py:467: LightningDeprecationWarning: Setting `Trainer(gpus='-1')` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices='-1')` instead.
  rank_zero_deprecation(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Loading pre-built model: https://github.com/weecology/DeepForest/releases/tag/1.0.0


In [22]:
start_time = time.time()
model.trainer.fit(model)
print(f"--- Training on CPU: {(time.time() - start_time):.2f} seconds ---")

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type      | Params
------------------------------------
0 | model | RetinaNet | 32.1 M
------------------------------------
31.9 M    Trainable params
222 K     Non-trainable params
32.1 M    Total params
128.592   Total estimated model params size (MB)


c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:366: UserWarning: One of given dataloaders is None and it will be skipped.
  rank_zero_warn("One of given dataloaders is None and it will be skipped.")
c:\Users\Robeck\Downloads\df\.venv\lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Epoch 4: 100%|██████████| 472/472 [02:25<00:00,  3.25it/s, loss=1.35] 

`Trainer.fit` stopped: `max_epochs=5` reached.


Epoch 4: 100%|██████████| 472/472 [02:25<00:00,  3.25it/s, loss=1.35]
--- Training on CPU: 681.41 seconds ---


In [30]:
torch.save(model.model.state_dict(),'model-500.pth', )

In [31]:
predicted_raster = model.predict_tile(src_img, return_plot = False, patch_size=500,patch_overlap=0.25)

  1%|▏         | 1872/130745 [03:32<4:04:34,  8.78it/s]